In [ ]:
import wget

url = 'https://arxiv.org/pdf/2311.12983'
filename = 'gaiapaper.pdf'
# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

file downloaded


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

filename = 'gaiapaper.pdf'
loader = PyPDFLoader(filename)
documents = loader.load()

In [17]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

24


In [20]:
texts[0].page_content[:1000]  # Display the first 1000 characters of the first chunk

'GAIA:\nA Benchmark for General AI Assistants\nGr´ egoire Mialon1, Cl´ ementine Fourrier2, Craig Swift3, Thomas Wolf2, Yann LeCun1, Thomas\nScialom4\n1FAIR, Meta, 2HuggingFace, 3AutoGPT, 4GenAI, Meta\nWe introduce GAIA, a benchmark for General AI Assistants that, if solved, would represent a\nmilestone in AI research. GAIA proposes real-world questions that require a set of fundamental\nabilities such as reasoning, multi-modality handling, web browsing, and generally tool-use proficiency.\nGAIA questions are conceptually simple for humans yet challenging for most advanced AIs: we\nshow that human respondents obtain 92% vs. 15% for GPT-4 equipped with plugins. This notable\nperformance disparity contrasts with the recent trend of LLMs outperforming humans on tasks\nrequiring professional skills in e.g. law or chemistry. GAIA’s philosophy departs from the current\ntrend in AI benchmarks suggesting to target tasks that are ever more difficult for humans. We posit\nthat the advent of Artif

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
model_path = "./models/"
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    cache_folder=model_path,
    model_kwargs={'device': 'cpu'}  # or 'cuda' if you have GPU
)

/tmp/ipykernel_2229421/3068215441.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
from langchain.vectorstores import Chroma
docsearch = Chroma.from_documents(texts, embedding_model)  # store the embedding in docsearch using Chromadb
print('document ingested')

/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


document ingested


In [25]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time, re

llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [29]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm_main, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True)
query = "What is GAIA in ten words?"
qa.invoke(query)

/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'query': 'What is GAIA in ten words?',
 'result': 'GAIA is a benchmark for general AI assistant evaluation systems.',
 'source_documents': [Document(metadata={'subject': '', 'total_pages': 24, 'source': 'gaiapaper.pdf', 'trapped': '/False', 'page': 17, 'creationdate': '2023-11-23T02:01:06+00:00', 'author': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'title': '', 'moddate': '2023-11-23T02:01:06+00:00', 'page_label': '18', 'keywords': ''}, page_content='C Extended description of GAIA\nDescription of capabilities. When answering the questions, annotators specified the steps that were followed\nand listed the tools they use. Based on the set of tools that were mentionned by the annotators, we defined\ncapabilities required by GAIA. For each capability, we report examples of corresponding tool as reported by\nannotators.\n• Web browsing: tools related to sear

In [30]:
query = "Is Le Cunn one of the authors?"
qa.invoke(query)

/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'query': 'Is Le Cunn one of the authors?',
 'result': 'Yes, Yann LeCun is listed as an author in the paper "Augmented language models: a survey" by Grégoire Mialon et al., 2023.',
 'source_documents': [Document(metadata={'creator': 'LaTeX with hyperref', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'page_label': '15', 'trapped': '/False', 'author': '', 'subject': '', 'creationdate': '2023-11-23T02:01:06+00:00', 'keywords': '', 'total_pages': 24, 'moddate': '2023-11-23T02:01:06+00:00', 'producer': 'pdfTeX-1.40.25', 'page': 14, 'title': '', 'source': 'gaiapaper.pdf'}, page_content='Melzi, Siva Reddy, Sneha Priscilla Makini, Soo-Hwan Lee, Spencer Torene, Sriharsha Hatwar, Stanislas Dehaene,\nStefan Divic, Stefano Ermon, Stella Biderman, Stephanie Lin, Stephen Prasad, Steven Piantadosi, Stuart Shieber,\nSummer Misherghi, Svetlana Kiritchenko, Swaroop Mishra, Tal Linzen, Tal Schuster, Tao Li, Tao Yu, Tariq Ali,\nTatsunori Hash

## Wrap into loop

In [31]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=llm_main, 
                                               chain_type="stuff", 
                                               retriever=docsearch.as_retriever(), 
                                               memory = memory, 
                                               get_chat_history=lambda h : h, 
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")
        
        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break
            
        result = qa({"question": query}, {"chat_history": history})
        
        history.append((query, result["answer"]))
        
        print("Answer: ", result["answer"])

In [32]:
qa()

/tmp/ipykernel_2229421/1813949221.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
/tmp/ipykernel_2229421/1813949221.py:22: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question": query}, {"chat_history": history})
/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:  How's it going? Is there something I can help you with or would you like to chat?


/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:  I'll do my best to provide quick answers by:

1. Understanding your question clearly and accurately
2. Referencing the provided context text, which is a detailed description of the ToolQA dataset and its creation process
3. Providing concise and direct answers based on the information in the context text
4. Not making assumptions or trying to fill in gaps in the information

If I'm unsure or don't know the answer to your question, I'll say so instead of providing an incorrect or speculative response.

Let's get started! What's your first question?


/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:  I'll do my best to provide concise answers, typically within a few sentences (around 10-20 words). If your question requires more explanation or context, I may provide additional information in the form of short paragraphs. However, I won't exceed a few hundred words per answer unless absolutely necessary.


/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:  I don't know. The text doesn't specify a word limit for answering questions.


/mnt/data3/rrao/projects/agentic-reasoning-framework/arf-env/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:  According to the text, GAIA (General AI Assistant) is a benchmark for evaluating AI systems that propose general assistant questions. It consists of 466 human-annotated questions designed to test various capabilities such as reasoning, multi-modality understanding, coding, and tool use. The goal of GAIA is to evaluate how well an AI system can perform real-world tasks by answering these questions in a zero-shot setting.
Answer: Goodbye!
